In [2]:
from pprint import pprint
# Required imports.
from main.abae.config import ABAEManagerConfig
from main.abae.model_manager import ABAEManager

In case we want to avoid the long gibberish by gensim on Word2Vec or other stuff:

In [3]:
import logging

logging.disable()
logging.disable(logging.DEBUG)
logging.disable(logging.INFO)

Let's make a first attempt on the various pre-processing pipelines:

### Full-reviews
Try to run the procedure on the default implementation of the pipeline. Default as it was thought to be a simple version for ABAE specifically.

## Preprocessing
Unlike LDA we should not toy too much with the sentence structure as ABAE uses word embeddings and needs the sequence information to weight the terms based on the surrounding context. One question remains:

**Should we work on sentence level or full reviews? Let's try a first simple comparison**

In [ ]:
corpus_path = "../dataset/output/default/pre_processed.80k.csv"
test_corpus_path = "../dataset/output/default/pre_processed.80k.test.csv"

manager = ABAEManager.from_scratch(ABAEManagerConfig('default'), corpus_path, override=True)
manager.train(corpus_path, verbose=2)
res = manager.evaluate([20, 10, 5, 3], test_corpus=test_corpus_path)

In [5]:
pprint(res)

{'coherence': [-9.519545495352105,
               -8.080626319870326,
               -7.4795967093038005,
               -6.558865921103352],
 'loss': [4.573703289031982, 4.572718143463135],
 'silhouette_score': 0.10651056468486786,
 'top': [20, 10, 5, 3]}


In [10]:
# See if lucky random initalization
corpus_path = "../dataset/output/default-8/pre_processed.80k.csv"
test_corpus_path = "../dataset/output/default-8/pre_processed.80k.test.csv"

manager = ABAEManager.from_scratch(ABAEManagerConfig('default-8'), corpus_path, override=True)
manager.train(corpus_path, verbose=2)
res = manager.evaluate([20, 10, 5, 3], test_corpus=test_corpus_path)

Pandas Apply:   0%|          | 0/60690 [00:00<?, ?it/s]

D:\PycharmProjects\nlp-course-project\.venv\Lib\site-packages\gensim\models\keyedvectors.py:551: UserWarning: Adding single vectors to a KeyedVectors which grows by one each time can be costly. Consider adding in batches or preallocating to the required size.
  warnings.warn(


Creating new Aspect embedding model

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/60690 [00:00<?, ?it/s]

Max sequence length calculation in progress...
We loose information on 1481(2.4402702257373536% of ds).
Generating a new compiled model from scratch
Training is starting:
Epoch 1/15
475/475 - 99s - 209ms/step - loss: 7.9819 - max_margin_loss: 7.9755
Epoch 2/15
475/475 - 97s - 203ms/step - loss: 4.9599 - max_margin_loss: 4.9572
Epoch 3/15
475/475 - 103s - 217ms/step - loss: 4.6922 - max_margin_loss: 4.6910
Epoch 4/15
475/475 - 105s - 221ms/step - loss: 4.6369 - max_margin_loss: 4.6377
Epoch 5/15
475/475 - 102s - 215ms/step - loss: 4.6034 - max_margin_loss: 4.6033
Epoch 6/15
475/475 - 108s - 227ms/step - loss: 4.5782 - max_margin_loss: 4.5744
Epoch 7/15
475/475 - 107s - 225ms/step - loss: 4.5533 - max_margin_loss: 4.5549
Epoch 8/15
475/475 - 104s - 218ms/step - loss: 4.5499 - max_margin_loss: 4.5480
Epoch 9/15
475/475 - 110s - 231ms/step - loss: 4.5308 - max_margin_loss: 4.5311
Epoch 10/15
475/475 - 113s - 237ms/step - loss: 4.5265 - max_margin_loss: 4.5252
Epoch 11/15
475/475 - 112s - 2

...

Max sequence length calculation in progress...
We loose information on 487(2.40731586752348% of ds).
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


Pandas Apply:   0%|          | 0/20230 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/20230 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/20230 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/20230 [00:00<?, ?it/s]

In [11]:
pprint(res)

{'coherence': [-9.532476935115621,
               -7.297630775527104,
               -6.547716215539502,
               -5.569106045492354],
 'loss': [4.510308265686035, 4.5080647468566895],
 'silhouette_score': 0.10978267341852188,
 'top': [20, 10, 5, 3]}


## Sentence-split reviews

In [2]:
corpus_path = "../dataset/output/default_sentences/pre_processed.80k.csv"
test_corpus_path = "../dataset/output/default_sentences/pre_processed.80k.test.csv"

In [ ]:
manager = ABAEManager.from_scratch(ABAEManagerConfig('sentence_default'), corpus_path)
manager.train(corpus_path, verbose=2)
res = manager.evaluate([20, 10, 5, 3], test_corpus=test_corpus_path)

In [4]:
pprint(res)

{'coherence': [-10.641038863629595,
               -9.138537658110502,
               -6.587250501495071,
               -5.5914570947321085],
 'loss': [5.903240203857422, 5.900863170623779],
 'silhouette_score': 0.20448735356330872,
 'top': [20, 10, 5, 3]}


In [45]:
corpus_path = "../dataset/output/default_sentences/pre_processed.80k.csv"
config = ABAEManagerConfig('sentence_default')
manager = ABAEManager.from_scratch(config, corpus_path)
manager.get_compiled_model(load_existing=True)
processor = manager.make_ev_processor(test_corpus_path)

aspects = processor.get_aspects(20)

Pandas Apply:   0%|          | 0/68390 [00:00<?, ?it/s]

Loading the existing found model as requested in path ./output/sentence_default/sentence_default.aspect_embeddings.model
Generating a new compiled model from fs


In [46]:
aspects

[[('games', tensor(0.4446, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('review', tensor(0.4162, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('price', tensor(0.4087, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('box', tensor(0.4049, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('copy', tensor(0.3943, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('designer', tensor(0.3898, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('rating', tensor(0.3842, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('star', tensor(0.3789, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('year', tensor(0.3781, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('criteria', tensor(0.3780, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('rate', tensor(0.3648, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('sell', tensor(0.3567, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('bgg', tensor(0.3552, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('friend', tensor(0.3546, device='cuda:0', grad_fn

In [48]:
manager.evaluate([20, 10, 5, 3], test_corpus=test_corpus_path)

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/77966 [00:00<?, ?it/s]

Max sequence length calculation in progress...
We loose information on 8(0.01026088294897776% of ds).
610/610 ━━━━━━━━━━━━━━━━━━━━ 161s 264ms/step - loss: 5.9393 - max_margin_loss: 5.9393
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/77966 [00:00<?, ?it/s]

Max sequence length calculation in progress...
We loose information on 8(0.01026088294897776% of ds).
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


Pandas Apply:   0%|          | 0/77966 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/77966 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/77966 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/77966 [00:00<?, ?it/s]

{'coherence': [-8.44545707953797,
  -7.3458720913356945,
  -5.829376841304451,
  -6.036148838026193],
 'top': [20, 10, 5, 3],
 'loss': [5.936696529388428, 5.937065601348877],
 'silhouette_score': 0.20680736005306244}

In [49]:
processor.get_aspects(20)

[[('games', tensor(0.4446, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('review', tensor(0.4162, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('price', tensor(0.4087, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('box', tensor(0.4049, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('copy', tensor(0.3943, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('designer', tensor(0.3898, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('rating', tensor(0.3842, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('star', tensor(0.3789, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('year', tensor(0.3781, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('criteria', tensor(0.3780, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('rate', tensor(0.3648, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('sell', tensor(0.3567, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('bgg', tensor(0.3552, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('friend', tensor(0.3546, device='cuda:0', grad_fn

I know that doing a  comparison on a single run is not that meaningful. <br>
I could do k-CV to estimate the expected model loss to get a valid analysis. <br>
But for the sake of the experiment we consider this good enough.

In [4]:
aspects

[[('games', tensor(0.4446, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('review', tensor(0.4162, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('price', tensor(0.4087, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('box', tensor(0.4049, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('copy', tensor(0.3943, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('designer', tensor(0.3898, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('rating', tensor(0.3842, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('star', tensor(0.3789, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('year', tensor(0.3781, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('criteria', tensor(0.3780, device='cuda:0', grad_fn=<SelectBackward0>))],
 [('dynasty', tensor(0.6239, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('stale', tensor(0.6182, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('forgivable', tensor(0.6125, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('blast', tensor(0.6099, device='cuda:

## NOUN only
I expect this to utterly fail in the task as we loose sentence structure which ABAE sounds to be abusing. <br>
Being based on word2vec embeddings this kind of pre-processing should be harmful I suppose.

In [5]:
from main.lda.pre_processing import extract_pos_ds
import pandas as pd

print("Creating the __noun filtered ds:")
corpus_path = "../dataset/output/pos_tagged/pre_processed.80k.csv"
store_path = "../dataset/output/pos_tagged/pre_processed.80k.noun_only.csv"
extract_pos_ds(pd.read_csv(corpus_path)['comments'], "__noun", store_path)
print("ds created under: " + store_path)

print("Creating the __noun filtered test ds:")
corpus_path = "../dataset/output/pos_tagged/pre_processed.80k.test.csv"
store_path = "../dataset/output/pos_tagged/pre_processed.80k.noun_only.test.csv"
extract_pos_ds(pd.read_csv(corpus_path)['comments'], "__noun", store_path)
print("ds created under: " + store_path)

Creating the __noun filtered ds:


Pandas Apply:   0%|          | 0/60701 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/60701 [00:00<?, ?it/s]

ds created under: ../dataset/output/pos_tagged/pre_processed.80k.noun_only.csv
Creating the __noun filtered test ds:


Pandas Apply:   0%|          | 0/20234 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/20234 [00:00<?, ?it/s]

ds created under: ../dataset/output/pos_tagged/pre_processed.80k.noun_only.test.csv


In [5]:
corpus_path = "../dataset/output/pos_tagged/pre_processed.80k.noun_only.csv"
test_corpus_path = "../dataset/output/pos_tagged/pre_processed.80k.noun_only.test.csv"

manager = ABAEManager.from_scratch(ABAEManagerConfig('noun_only'), corpus_path)
manager.train(corpus_path, verbose=2)
res = manager.evaluate([20, 10, 5, 3], test_corpus=test_corpus_path)

Pandas Apply:   0%|          | 0/51995 [00:00<?, ?it/s]

D:\PycharmProjects\nlp-course-project\.venv\Lib\site-packages\gensim\models\keyedvectors.py:551: UserWarning: Adding single vectors to a KeyedVectors which grows by one each time can be costly. Consider adding in batches or preallocating to the required size.
  warnings.warn(


Creating new Aspect embedding model

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/51995 [00:00<?, ?it/s]

Max sequence length calculation in progress...
We loose information on 191(0.3673430137513222% of ds).
Generating a new compiled model from scratch
Training is starting:
Epoch 1/15
407/407 - 101s - 249ms/step - loss: 9.0340 - max_margin_loss: 9.0234
Epoch 2/15
407/407 - 102s - 251ms/step - loss: 5.7994 - max_margin_loss: 5.7975
Epoch 3/15
407/407 - 101s - 247ms/step - loss: 5.6593 - max_margin_loss: 5.6584
Epoch 4/15
407/407 - 93s - 228ms/step - loss: 5.6012 - max_margin_loss: 5.6011
Epoch 5/15
407/407 - 96s - 235ms/step - loss: 5.5773 - max_margin_loss: 5.5765
Epoch 6/15
407/407 - 91s - 225ms/step - loss: 5.5658 - max_margin_loss: 5.5645
Epoch 7/15
407/407 - 91s - 224ms/step - loss: 5.5454 - max_margin_loss: 5.5439
Epoch 8/15
407/407 - 91s - 223ms/step - loss: 5.5357 - max_margin_loss: 5.5339
Epoch 9/15
407/407 - 93s - 228ms/step - loss: 5.5175 - max_margin_loss: 5.5152
Epoch 10/15
407/407 - 89s - 220ms/step - loss: 5.5071 - max_margin_loss: 5.5060
Epoch 11/15
407/407 - 89s - 219ms/st

...

Max sequence length calculation in progress...
We loose information on 87(0.4906107257655219% of ds).
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


Pandas Apply:   0%|          | 0/17733 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/17733 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/17733 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/17733 [00:00<?, ?it/s]

In [6]:
pprint(res)

{'coherence': [-8.948873045807769,
               -7.445881505510981,
               -6.305143535846072,
               -5.799752060521229],
 'loss': [5.478001117706299, 5.478464126586914],
 'silhouette_score': 0.1511905938386917,
 'top': [20, 10, 5, 3]}


For the future work and as one by the proposed ABAE paper we won't be splitting up reviews in sentences but use the full review as the model does not increase much in performance if not done like this.

## Hyperparameters tuning results

In [12]:
import json
import pandas as pd

file_path = "./output/config/abae_configurations_results.json"
configs = [
    pd.DataFrame(json.load(open(file_path))).at[11, 'config'],
    pd.DataFrame(json.load(open(file_path))).at[20, 'config'],
    # These two perform worse but have a lower variance which might result in a more robust solution
    pd.DataFrame(json.load(open(file_path))).at[14, 'config'],
    pd.DataFrame(json.load(open(file_path))).at[15, 'config'],
]

runs = [
    dict(config=ABAEManagerConfig.from_configuration(f"final_full_{i}", configs[i]), results=[], model=None)
    for i in range(len(configs))
]

Run on configs

In [13]:
corpus = "../dataset/output/default/pre_processed.310k.csv"
test_corpus_path = "../dataset/output/default/pre_processed.310k.test.csv"

In [ ]:
for run in runs:
    manager = ABAEManager.from_scratch(run['config'], corpus, override=True)
    manager.train(corpus, verbose=2)
    run['results'] = manager.evaluate([20, 10, 5, 3], test_corpus=test_corpus_path)
    run['model'] = manager.get_compiled_model(load_existing=True)

    # Still print the results in output.
    pprint(run['results'])

In [17]:
[run['results'] for run in
 runs]  # The second configuration yielded the overall best model. The first one seems to be overfitting

[{'coherence': [-12.85030957360423,
   -11.720057513868097,
   -11.361470141436131,
   -11.536755467282116],
  'top': [20, 10, 5, 3],
  'loss': [3.9788215160369873, 3.977562665939331],
  'silhouette_score': 0.00810018740594387},
 {'coherence': [-12.968785972106792,
   -11.427717010393682,
   -11.386505234330448,
   -10.055480345568363],
  'top': [20, 10, 5, 3],
  'loss': [4.198480129241943, 4.197236061096191],
  'silhouette_score': 0.04261905327439308},
 {'coherence': [-13.761206825132213,
   -14.066563112308721,
   -13.169986067132188,
   -14.055746424933238],
  'top': [20, 10, 5, 3],
  'loss': [5.173580169677734, 5.172301769256592],
  'silhouette_score': 0.05377541854977608},
 {'coherence': [-14.414729002014086,
   -13.55967317937999,
   -11.93764523613414,
   -10.512464698706138],
  'top': [20, 10, 5, 3],
  'loss': [4.7525739669799805, 4.751503944396973],
  'silhouette_score': -0.03403297811746597}]

In [23]:
best_run = runs[1]
manager = ABAEManager.from_scratch(best_run['config'], corpus)
manager.get_compiled_model(load_existing=True)
ev_processor = manager.make_ev_processor(test_corpus_path)
aspects = ev_processor.get_aspects(20)

Pandas Apply:   0%|          | 0/233384 [00:00<?, ?it/s]

Loading the existing found model as requested in path ./output/final_full_1/final_full_1.aspect_embeddings.model
Generating a new compiled model from fs


In [24]:
aspects

[[('daughter', tensor(0.6013, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('friend', tensor(0.5799, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('son', tensor(0.5796, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('mom', tensor(0.5775, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('request', tensor(0.5766, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('month', tensor(0.5754, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('weekend', tensor(0.5648, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('dad', tensor(0.5610, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('week', tensor(0.5514, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('anymore', tensor(0.5486, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('happily', tensor(0.5486, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('regularly', tensor(0.5450, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('husband', tensor(0.5362, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('niece', tensor(0.5326, device='cuda:

In [19]:
ev_processor.u_mass_coherence_model(3).get_coherence_per_topic()

Pandas Apply:   0%|          | 0/77794 [00:00<?, ?it/s]

[-3.9255778188673314,
 -9.682578001477758,
 -7.920850829364216,
 -20.800788480558847,
 -14.668867895648573,
 -4.428323920088644,
 -4.3330299233703995,
 -4.018951870591462,
 -9.562629431391636,
 -11.194518106501354,
 -19.603888372171728,
 -8.714951871114133,
 -19.654037319663455,
 -4.120865113649609,
 -9.26197477529872,
 -14.944052565152163,
 -4.107279579752117]

In [25]:
aspects[3]  # Seems to be talking about games

[('sequel', tensor(0.7444, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('iteration', tensor(0.7227, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('imperium', tensor(0.6948, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('successor', tensor(0.6804, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('reimplementation',
  tensor(0.6739, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('evolution', tensor(0.6226, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('remake', tensor(0.6039, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('og', tensor(0.5992, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('battlelore', tensor(0.5919, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('trilogy', tensor(0.5778, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('spiritual', tensor(0.5722, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('argentina', tensor(0.5713, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('vastly', tensor(0.5685, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('standalone', tenso

In [26]:
aspects[4]  # Components quality

[('packaging', tensor(0.7969, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('sturdy', tensor(0.7592, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('cardstock', tensor(0.7442, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('plastic', tensor(0.7255, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('inlay', tensor(0.7232, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('cardboard', tensor(0.6974, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('figurine', tensor(0.6960, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('insert', tensor(0.6918, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('thick', tensor(0.6829, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('minis', tensor(0.6821, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('tin', tensor(0.6811, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('standee', tensor(0.6792, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('components', tensor(0.6753, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('flimsy', tensor(0.6544, device='cuda:

In [28]:
aspects[-2]  # Role of the component/mechanics role

[('zombie', tensor(0.7120, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('survivor', tensor(0.6937, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('demon', tensor(0.6657, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('fighter', tensor(0.6640, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('horde', tensor(0.6423, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('boss', tensor(0.6349, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('villain', tensor(0.6231, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('tale', tensor(0.6222, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('nemesis', tensor(0.6179, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('battler', tensor(0.6104, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('baddie', tensor(0.6096, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('survival', tensor(0.6046, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('minion', tensor(0.6037, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('swarm', tensor(0.6030, device='cuda:0', grad_f

In [29]:
aspects[-5]  # ??? Seems to be associated to game-names/abbreviations

[('princes', tensor(0.6844, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('el', tensor(0.6561, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('la', tensor(0.6479, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('isle', tensor(0.6314, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('fury', tensor(0.6311, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('<game_name>', tensor(0.6130, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('de', tensor(0.6121, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('feast', tensor(0.6090, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('dorado', tensor(0.6090, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('et', tensor(0.6022, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('goa', tensor(0.6022, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('und', tensor(0.5964, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('le', tensor(0.5957, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('mal', tensor(0.5826, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('h',

In [20]:
ev_processor.u_mass_coherence_model(5).get_coherence_per_topic()

Pandas Apply:   0%|          | 0/77794 [00:00<?, ?it/s]

[-3.9255778188673314,
 -9.682578001477758,
 -7.920850829364216,
 -20.800788480558847,
 -14.668867895648573,
 -4.428323920088644,
 -4.3330299233703995,
 -4.018951870591462,
 -9.562629431391636,
 -11.194518106501354,
 -19.603888372171728,
 -8.714951871114133,
 -19.654037319663455,
 -4.120865113649609,
 -9.26197477529872,
 -14.944052565152163,
 -4.107279579752117]

In [21]:
ev_processor.u_mass_coherence_model(10).get_coherence_per_topic()

Pandas Apply:   0%|          | 0/77794 [00:00<?, ?it/s]

[-3.9255778188673314,
 -9.682578001477758,
 -7.920850829364216,
 -20.800788480558847,
 -14.668867895648573,
 -4.428323920088644,
 -4.3330299233703995,
 -4.018951870591462,
 -9.562629431391636,
 -11.194518106501354,
 -19.603888372171728,
 -8.714951871114133,
 -19.654037319663455,
 -4.120865113649609,
 -9.26197477529872,
 -14.944052565152163,
 -4.107279579752117]

In [22]:
ev_processor.u_mass_coherence_model(20).get_coherence_per_topic()

Pandas Apply:   0%|          | 0/77794 [00:00<?, ?it/s]

[-11.292684969558596,
 -13.340562485156116,
 -17.25537068228306,
 -16.67731065741633,
 -12.729347415810782,
 -6.863208686903573,
 -12.21684293614156,
 -9.525660566176187,
 -9.072414689878123,
 -13.020833492935134,
 -17.018505533620598,
 -15.922856249580782,
 -17.517073871647344,
 -12.29556240725517,
 -9.56280301759823,
 -15.131052446773475,
 -11.027271417080396]

In [ ]:
config = ABAEManagerConfig('final_full_1')

In [39]:
best_run = runs[1]
manager = ABAEManager.from_scratch(config, corpus_path)
manager.get_compiled_model(load_existing=True)
processor = manager.make_ev_processor(test_corpus_path)

aspects = processor.get_aspects(15)

Pandas Apply:   0%|          | 0/233897 [00:00<?, ?it/s]

Loading the existing found model as requested in path ./output/final_full_1/final_full_1.aspect_embeddings.model
Generating a new compiled model from fs


In [43]:
aspects = processor.get_aspects(20)
aspects

[[('mode', tensor(0.5173, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('solo', tensor(0.5006, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('varied', tensor(0.4984, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('progression', tensor(0.4800, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('differently', tensor(0.4707, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('variability', tensor(0.4523, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('challenging', tensor(0.4435, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('campaign', tensor(0.4395, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('variety', tensor(0.4295, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('enhance', tensor(0.4290, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('scenario', tensor(0.4259, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('bot', tensor(0.4176, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('evolve', tensor(0.4160, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('improve', tenso

In [ ]:
corpus_path = "../dataset/output/default_sentences/pre_processed.310k.csv"
test_corpus_path = "../dataset/output/default_sentences/pre_processed.310k.test.csv"

manager = ABAEManager.from_scratch(runs[0]['config'], corpus_path)
manager.train(corpus_path, verbose=2)
res = manager.evaluate([20, 10, 5, 3], test_corpus=test_corpus_path)